In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [2]:
print(tf.__version__)

2.10.0


In [1]:
from refractio import snowflake
petrol_cons = snowflake.get_dataframe("APPLICABLE_ROLES")

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Connection details fetched: {'params': {'READER': {'schema': 'INFORMATION_SCHEMA', 'cloudPlatform': 'azure', 'role': '"ACCOUNTADMIN"', 'clientId': None, 'tokenUrl': None, 'scopeUrl': None, 'type': 'RDBMS', 'wareHouse': 'COMPUTE_WH', 'accountId': 'ib37295', 'privateKey': None, 'password': 'Fosfor@2024', 'database': 'FDC_INTEGRATION', 'tables': 'APPLICABLE_ROLES', 'sub_type': 'SNOWFLAKE', 'subType': 'SNOWFLAKE', 'clientSecret': None, 'authenticationType': 'dbPassword', 'rowCount': 10, 'region': 'central-india', 'user': 'rakeshgadiparthi', 'applicationName': 'FOSFOR', 'passPhrase': None}}, 'url': 'http://connectors-backend-service/connectors/v1/connectors/metadata', 'access': True, 'errorMsg': None, 'values': {}}
Ex: 250001: Could not connect to Snowflake backend after 2 attempt(s).Aborting
Exception occ

In [4]:
# petrol_cons = pd.read_csv('/data/petrol_consumption.csv')

In [5]:
petrol_cons.head()

,PETROL_TAX,AVERAGE_INCOME,PAVED_HIGHWAYS,POPULATION_DRIVER_LICENCE,PETROL_CONSUMPTION
0,6.00,3921,7949,0.6955,657.0
1,6.58,5429,6428,0.6073,694.0
2,7.00,4085,17751,0.5526,341.0
3,7.50,5151,14041,0.7561,585.0
4,8.00,4287,7966,0.4824,381.0


In [6]:
X = petrol_cons.iloc[:, 0:4].values
y = petrol_cons.iloc[:, 4].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model

In [8]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(100, activation='relu')(input_layer)
dense_layer_2 = Dense(50, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(25, activation='relu')(dense_layer_2)
output = Dense(1)(dense_layer_3)

model_tf = Model(inputs=input_layer, outputs=output)
model_tf.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

In [9]:
print(model_tf.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 100)               500       
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 25)                1275      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 6,851
Trainable params: 6,851
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
history = model_tf.fit(X_train, y_train, batch_size=8, epochs=1, verbose=1, validation_split=0.2)

80/80 [==============================] - 1s 3ms/step - loss: 450368.5000 - mean_squared_error: 450368.5000 - val_loss: 451761.5625 - val_mean_squared_error: 451761.5625


In [11]:
history = model_tf.fit(X_train, y_train, batch_size=8, epochs=1, verbose=1, validation_split=0.2)

80/80 [==============================] - 0s 2ms/step - loss: 289083.5312 - mean_squared_error: 289083.5312 - val_loss: 102128.6484 - val_mean_squared_error: 102128.6484


In [12]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred_train = model_tf.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred = model_tf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred)))

25/25 [==============================] - 0s 738us/step
307.21384854474735
7/7 [==============================] - 0s 882us/step
302.6351658186698


In [13]:
cols = ['Petrol_tax','Average_income','Paved_Highways','Population_Driver_licence(%)']

In [14]:
#Model function
def model(algorithm,dtrain_X,dtrain_Y,dtest_X,dtest_Y,cols=None):

    #algorithm.fit(dtrain_X,dtrain_Y)
    predictions = algorithm.predict(dtest_X)
    prediction_probabilities = algorithm.predict(dtest_X)
    print (algorithm)
    
    return predictions,prediction_probabilities,algorithm

In [15]:
y_pred,y_prob,model_obj=model(model_tf,X_train,y_train,X_test,y_test)

7/7 [==============================] - 0s 1ms/step


In [16]:
from refractml import *
from refractml.constants import MLModelFlavours

/tmp/pip_packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [17]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = model.predict(data)
    #prediction = pd.Series(model.predict(data))
    #print (prediction)
    return prediction
    #return prediction.tolist()

In [18]:
petrol_cons.head()

,PETROL_TAX,AVERAGE_INCOME,PAVED_HIGHWAYS,POPULATION_DRIVER_LICENCE,PETROL_CONSUMPTION
0,6.00,3921,7949,0.6955,657.0
1,6.58,5429,6428,0.6073,694.0
2,7.00,4085,17751,0.5526,341.0
3,7.50,5151,14041,0.7561,585.0
4,8.00,4287,7966,0.4824,381.0


In [19]:
X_temp = pd.DataFrame(X,columns=cols)

In [20]:
import requests
req = requests.Request()
req.json = {"payload":X_temp.head(1).to_json(orient = 'records')}

In [21]:
req.json['payload']

'[{"Petrol_tax":6.0,"Average_income":3921.0,"Paved_Highways":7949.0,"Population_Driver_licence(%)":0.6955}]'

In [22]:
sch = generate_schema(score,(model_obj, req),X_temp)
print(sch)

1/1 [==============================] - 0s 50ms/step
{'input': {'type': 'pd.core.frame.DataFrame', 'items': {'type': 'object', 'properties': {'Petrol_tax': {'type': 'number', 'format': 'double'}, 'Average_income': {'type': 'number', 'format': 'double'}, 'Paved_Highways': {'type': 'number', 'format': 'double'}, 'Population_Driver_licence(%)': {'type': 'number', 'format': 'double'}}}, 'example': [{'Petrol_tax': 6.0, 'Average_income': 3921.0, 'Paved_Highways': 7949.0, 'Population_Driver_licence(%)': 0.6955}, {'Petrol_tax': 6.58, 'Average_income': 5429.0, 'Paved_Highways': 6428.0, 'Population_Driver_licence(%)': 0.6073}]}, 'output': {'type': 'np.ndarray', 'items': {'type': 'array', 'items': {'type': 'number', 'format': 'float'}}, 'example': [[1157630.5]]}}


In [23]:
register_model(model_obj, score, name="CarEvaluation_RegressionTENSORFLOW", 
                description="Classify Car basis the feature",
                flavour=MLModelFlavours.tensorflow, schema=sch,
                y_true=y_test, y_pred = y_pred, #y_pred=pd.Series(y_pred), 
                prob=y_pred, features=cols, 
               labels=["Petrol_Consumption"], 
               init_script="" ,
                model_type="regression", 
                input_type="json", explain_ai=True, x_train=X_train, x_test=X_test, 
                y_train=y_train, y_test=y_test,
                feature_names=cols,feature_ids=cols,
                target_names=["Petrol_Consumption"], 
                kyd=False, kyd_score =False)
print("Registering model")

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


Registering model
